In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  mean_absolute_error

In [50]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              15000 non-null  int64  
 1   Sex             15000 non-null  object 
 2   Length          15000 non-null  float64
 3   Diameter        15000 non-null  float64
 4   Height          15000 non-null  float64
 5   Weight          15000 non-null  float64
 6   Shucked Weight  15000 non-null  float64
 7   Viscera Weight  15000 non-null  float64
 8   Shell Weight    15000 non-null  float64
 9   Age             15000 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.1+ MB


In [51]:
size_mapping = {'M' : 1, 'F' : 2}
df_train['Sex'] = df_train['Sex'].map(size_mapping)
df_test['Sex'] = df_test['Sex'].map(size_mapping)

In [52]:
df_train.describe()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,15000.000000,9975.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,1.455739,1.310631,1.018376,0.345566,23.069466,9.960247,4.991979,6.612909,9.951200
std,4330.271354,0.498062,0.291486,0.240428,0.091620,12.692168,5.634580,2.805122,3.585832,3.223028
min,0.000000,1.000000,0.262500,0.000000,0.000000,0.056699,0.028349,0.014175,0.085048,1.000000
25%,3749.750000,1.000000,1.150000,0.875000,0.287500,13.083294,5.584852,2.778251,3.827183,8.000000
50%,7499.500000,1.000000,1.375000,1.075000,0.362500,23.440764,9.823102,4.890289,6.803880,10.000000
75%,11249.250000,2.000000,1.525000,1.200000,0.412500,32.034935,13.877080,6.959802,9.029316,11.000000
max,14999.000000,2.000000,1.950000,1.700000,0.650000,80.101512,35.521923,16.300963,27.839209,29.000000


In [53]:
df_train.isna().sum()

id                   0
Sex               5025
Length               0
Diameter             0
Height               0
Weight               0
Shucked Weight       0
Viscera Weight       0
Shell Weight         0
Age                  0
dtype: int64

In [54]:
imr = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imr_data = imr.fit_transform(df_train[['Sex']].values)
imr_data

array([[1.],
       [2.],
       [1.],
       ...,
       [1.],
       [1.],
       [2.]])

In [55]:
df_train[['Sex']] = imr_data
df_train['Sex']

0        1.0
1        2.0
2        1.0
3        1.0
4        1.0
        ... 
14995    1.0
14996    1.0
14997    1.0
14998    1.0
14999    2.0
Name: Sex, Length: 15000, dtype: float64

In [56]:
imr = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imr_data = imr.fit_transform(df_test[['Sex']].values)
imr_data

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [57]:
df_test[['Sex']] = imr_data
df_test['Sex']

0       1.0
1       1.0
2       1.0
3       2.0
4       1.0
       ... 
9995    1.0
9996    1.0
9997    1.0
9998    1.0
9999    1.0
Name: Sex, Length: 10000, dtype: float64

In [58]:
X = df_train.drop([ 'Age'], axis=1)
y = df_train['Age']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X.values,y,
                                                    test_size=0.5,
                                                    random_state=45)

In [60]:
regressor = make_pipeline(PolynomialFeatures(2),LinearRegression())# StandardScaler(),
regressor.fit(X_train, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [61]:
score = regressor.score(X_test, y_test)
print('Score: ', score)
print('Accuracy: ' + str(score*100) + '%')

Score:  0.6196843868449962
Accuracy: 61.96843868449962%


In [62]:
y_pred_ = regressor.predict(X_test)
print(f'MAE = {mean_absolute_error(y_test, y_pred_)}')

df_test.shape[0]

MAE = 1.3932292405111764


10000

In [63]:
y_pred = regressor.predict(df_test)
df_y_pred = pd.Series(y_pred)
df_id = pd.Series(df_test["id"])
df = pd.DataFrame({'id': df_id, 'Age': df_y_pred})
df['Age'].round(0)
df['Age'] = df['Age'].astype(int) 
df


c:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


,id,Age
0,15000,11
1,15001,4
2,15002,7
3,15003,11
4,15004,11
...,...,...
9995,24995,7
9996,24996,4
9997,24997,6
9998,24998,7


In [64]:
df.to_csv('Submission.csv', index=False)  